In [49]:
import librosa as lb
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
from numpy.core.shape_base import block
from scipy import signal
from scipy import stats
import statistics
import os
import glob
import csv
import pandas as pd
from scipy.io import wavfile
import math
import soundfile as sf
import copy

import pywt
from pywt import wavedec


# from HW
def FeatureTimeRms(xb):
    # number of results
    numBlocks = xb.shape[0]
    # allocate memory
    vrms = np.zeros(numBlocks)
    for n in range(0, numBlocks):
        # calculate the rms
        vrms[n] = np.sqrt(np.dot(xb[n,:], xb[n,:]) / xb.shape[1])
    # convert to dB
    epsilon = 1e-5  # -100dB
    vrms[vrms < epsilon] = epsilon
    vrms = 20 * np.log10(vrms)
    return (vrms)

# from HW
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    #compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs    
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]   
    return (xb,t)

# from HW
def comp_acf(inputVector, bIsNormalized = True):
    if bIsNormalized:
        norm = np.dot(inputVector, inputVector)
    else:
        norm = 1
    afCorr = np.correlate(inputVector, inputVector, "full") / norm
    afCorr = afCorr[np.arange(inputVector.size-1, afCorr.size)]
    return (afCorr)

def onset_detection(xb, t, thresh=0.5):
    # RMS
    rms = FeatureTimeRms(xb)
    # envelope
    B, A = signal.butter(2, 0.1)
    env = signal.filtfilt(B, A, rms)
    diff = np.diff(env)
    # threshold
    mask = diff < thresh
    diff[mask] = 0
    peaks = lb.util.peak_pick(diff, 50,50,10,10,0,50)
    peaks_t = t[peaks]

    return peaks_t

def IOI_histogram(peaks, bins):
    ioi = np.diff(peaks)
    return np.histogram(ioi, bins)

def get_tempo_v1(data, fs):
    xb, t = block_audio(data, 1024, 512, fs)
    peaks = onset_detection(xb, t)
    hist = IOI_histogram(peaks, 10)
    mode_interval = hist[1][np.argmax(hist[0])]
    bpm = 60 / mode_interval
    return bpm

In [36]:
df = pd.read_csv('/Users/roseqsun/Desktop/ACA Final/manban_data.csv')
manbanfiles = df['File'].tolist()
manbandirectory = []
for filename in manbanfiles:
    manbandirectory.append("/Users/roseqsun/Desktop/ACA Final/JARC/" + filename)

In [37]:
df = pd.read_csv('/Users/roseqsun/Desktop/ACA Final/zhongsanyan_data.csv')
zhongsanyanfiles = df['File'].tolist()
zhongsanyandirectory = []
for filename in zhongsanyanfiles:
    zhongsanyandirectory.append("/Users/roseqsun/Desktop/ACA Final/JARC/" + filename)

In [38]:
df = pd.read_csv('/Users/roseqsun/Desktop/ACA Final/kuaisanyan_data.csv')
kuaisanyanfiles = df['File'].tolist()
kuaisanyandirectory = []
for filename in kuaisanyanfiles:
    kuaisanyandirectory.append("/Users/roseqsun/Desktop/ACA Final/JARC/" + filename)

In [39]:
df = pd.read_csv('/Users/roseqsun/Desktop/ACA Final/yuanban_data.csv')
yuanbanfiles = df['File'].tolist()
yuanbandirectory = []
for filename in yuanbanfiles:
    yuanbandirectory.append("/Users/roseqsun/Desktop/ACA Final/JARC/" + filename)

In [40]:
df = pd.read_csv('/Users/roseqsun/Desktop/ACA Final/erliu_data.csv')
erliufiles = df['File'].tolist()
erliudirectory = []
for filename in erliufiles:
    erliudirectory.append("/Users/roseqsun/Desktop/ACA Final/JARC/" + filename)

In [41]:
df = pd.read_csv('/Users/roseqsun/Desktop/ACA Final/liushui_data.csv')
liushuifiles = df['File'].tolist()
liushuidirectory = []
for filename in liushuifiles:
    liushuidirectory.append("/Users/roseqsun/Desktop/ACA Final/JARC/" + filename)

In [42]:
df = pd.read_csv('/Users/roseqsun/Desktop/ACA Final/kuaiban_data.csv')
kuaibanfiles = df['File'].tolist()
kuaibandirectory = []
for filename in kuaibanfiles:
    kuaibandirectory.append("/Users/roseqsun/Desktop/ACA Final/JARC/" + filename)

In [50]:
def detect_tempo(directory):
    data, samplerate = sf.read(directory)
    midpoint = int(len(data)/ 2)
    sect = 15 * samplerate
    data = data[midpoint - sect : midpoint + sect]
    data = data[:,0]
    tempo = get_tempo_v1(data,samplerate)
    return tempo

In [51]:
def get_tempo_data(path):
    tempo_data_list = []
    for directory in path:
        tempo_data_list.append(detect_tempo(directory))
    return tempo_data_list

In [58]:
a = get_tempo_data(manbandirectory)
manban_df = pd.DataFrame(a,columns=['Tempo'])
manban_df.to_csv('manban_tempo.csv',index=False)

In [53]:
a = get_tempo_data(manbandirectory)
b = get_tempo_data(zhongsanyandirectory)
c = get_tempo_data(kuaisanyandirectory)
manban_df = pd.DataFrame(a,columns=['Tempo'])
zhongsanyan_df = pd.DataFrame(b,columns=['Tempo'])
kuaisanyan_df = pd.DataFrame(b,columns=['Tempo'])
sanyan = [manban_df, zhongsanyan_df, kuaisanyan_df]
sanyan_df = pd.concat(sanyan)
sanyan_df.to_csv('sanyan_tempo.csv',index=False)

KeyboardInterrupt: 

In [54]:
a = get_tempo_data(yuanbandirectory)
yuanban_df = pd.DataFrame(a,columns=['Tempo'])
yuanban_df.to_csv('yuanban_tempo.csv',index=False)

In [59]:
a = get_tempo_data(erliudirectory)
erliu_df = pd.DataFrame(a,columns=['Tempo'])
erliu_df.to_csv('erliu_tempo.csv',index=False)

In [56]:
a = get_tempo_data(liushuidirectory)
liushui_df = pd.DataFrame(a,columns=['Tempo'])
liushui_df.to_csv('liushui_tempo.csv',index=False)

In [57]:
a = get_tempo_data(kuaibandirectory)
kuaiban_df = pd.DataFrame(a,columns=['Tempo'])
kuaiban_df.to_csv('kuaiban_tempo.csv',index=False)